#### 1. Download MODIS .hdf files for Aqua (MYD) and Terra (MOD)

In [ ]:
# run the following command on a bash terminal
# replace DATA_PATH, KEY, and TARGET_PATH
# wget -e robots=off -m -np -R .html,.tmp -nH --cut-dirs=3 "https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/DATA_PATH" --header "Authorization: Bearer KEY" -P TARGET_PATH

#### 2. Convert .hdf to daily .nc

In [145]:
from glob import glob
from pyhdf.SD import SD, SDC
import numpy as np
import xarray as xr
from pyresample import geometry, bilinear
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from matplotlib import colors

In [11]:
year = '2019'
satellite = 'MYD04_L2' # 'MOD04_L2' / 'MYD04_L2'

if int(year) in [2000, 2004, 2008, 2012, 2016]:
    days_annual = 366
else:
    days_annual = 365
    
if satellite == 'MOD04_L2':
    if int(year) == 2000:
        days_begin = 55
    else:
        days_begin = 1
        
if satellite == 'MYD04_L2':
    if int(year) == 2002:
        days_begin = 185
    else:
        days_begin = 1
        
juldays = (["%.3d" % i for i in range(days_begin, days_annual + 1)])

In [23]:
# area setup - global
area_id = 'global'
description = 'MODIS AOD lat lon grid'
projection_id = 'global'
x_size = 3600 # pixels for global 0.1 degree
y_size = 1800 # pixels for global 0.1 degree
projection_dict = {'units': 'degrees', 'proj': 'eqc'} # proj eqc is regular lat lon
area_extent = (-20037508.34, -10018754.17, 20037508.34, 10018754.17) # x_ll, y_ll, x_ur, y_ur for regular lat lon
area_def = geometry.AreaDefinition(
    area_id,
    description,
    projection_id,
    projection_dict,
    x_size,
    y_size,
    area_extent
)

In [ ]:
global_reference_file = xr.open_dataset('/nfs/a336/earlacoa/modis/merged_files/nc_annual/Merged_MOD_MYD_04_L2_C061_2009_annual.nc')
global_reference_lat = global_reference_file['latitude']
global_reference_lon = global_reference_file['longitude']

In [ ]:
for julday in juldays:
    index = 0 
    hdf_files = []
    path = '/nfs/a336/earlacoa/modis/' + satellite + '.061/' + satellite + '/
    hdf_files.extend(glob(path + year + '/' + julday + '/*hdf'))
    
    for file in hdf_files:
        reader = open(file)
        hdf_file = SD(file, SDC.READ)

        data_raw = hdf_file.select('AOD_550_Dark_Target_Deep_Blue_Combined')
        data = data_raw[:,:].astype(np.double)
        
        latitude = hdf_file.select('Latitude')[:,:]
        longitude = hdf_file.select('Longitude')[:,:]
        
        attributes = data_raw.attributes(full=1)
        add_offset = attributes["add_offset"][0]
        fill_value = attributes["_FillValue"][0]
        scale_factor = attributes["scale_factor"][0]      
        units = attributes["units"][0]

        data[data == fill_value] = np.nan
        data = (data - add_offset) * scale_factor 
        data_masked = np.ma.masked_array(data, np.isnan(data))
        
        if index == 0 :
            data_julday = data_masked
            latitude_julday = latitude
            longitude_julday = longitude
        else:
            data_julday = np.vstack([data_julday, data_masked])
            latitude_julday = np.vstack([latitude_julday, latitude])
            longitude_julday = np.vstack([longitude_julday, longitude])
        index += 1
        
    longitude_julday[longitude_julday == -999.0] = np.nan
    latitude_julday[latitude_julday == -999.0] = np.nan
    
    # swath setup
    swath_def = geometry.SwathDefinition(
        lons=longitude_julday,
        lats=latitude_julday
    )
    
    # swath resampling
    # - bilinear interpolation for irregular swath grids
    # - smoother results near swath edges
    result = bilinear.resample_bilinear(
        data_julday, 
        swath_def, 
        area_def, 
        radius=50e3, 
        neighbours=32,         
        nprocs=1, 
        fill_value=0, 
        reduce_data=True, 
        segments=None, 
        epsilon=0
    )
    
    result[result <= 0.0] = np.nan
    result = np.ma.masked_invalid(result)
    
    # create dataset for .nc
    ds = xr.DataArray(
        data = np.flipud(result), 
        coords = [global_reference_lat, global_reference_lon],
        dims = ['lat', 'lon']
    )
    ds.name = 'AOD'
    ds = ds.assign_coords({'time': datetime.strptime(year[-2:] + julday, '%y%j')})
    ds.to_netcdf(path + 'nc/' + satellite + '_C061_' + year + '_' + julday + '.nc')

    # plot
    aod = ds.values
    xx, yy = np.meshgrid(ds.lon.values, ds.lat.values)
    
    fig = plt.figure(1, figsize=(5, 5))
    gs = gridspec.GridSpec(1, 1)
    ax = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
    ax.set_extent([-180, 180, -60, 85], crs=ccrs.PlateCarree())
    levels = (0, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,100000)
    norm = colors.Normalize(vmin=levels[0], vmax=levels[-2])
    cmap = colors.ListedColormap(['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#bd0026','#800026'])
    im = ax.contourf(xx, yy, conc, levels, cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    plt.annotate(r'\textbf{' + chr(97 + index) + '}', xy=(0,1.05), xycoords='axes fraction', fontsize=14, weight='bold')
    plt.title(target, fontsize=14)
    sm = plt.cm.ScalarMappable(norm=norm, cmap=im.cmap)
    sm.set_array([])  
    cb = plt.colorbar(sm, fraction=0.035, norm=norm, cmap=cmap, ticks=im.levels, format=fmt)
    cb.set_label(cb_label, size=14)
    cb.ax.tick_params(labelsize=14)
    
    
    cmap = 'YlOrRd'
    norm = Normalize(vmin=0, vmax=1)
    im = map.contourf(xx, yy, aod, (0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1000), cmap=cmap, norm=norm)
    sm = plt.cm.ScalarMappable(norm=norm, cmap=im.cmap)
    sm.set_array([])
    cb = map.colorbar(sm, "right", size="5%", pad='2%', norm=norm, cmap=cmap, ticks=(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0))
    cb.set_label('AOD 550 nm Dark Target Deep Blue Combined', size=12)
    plt.title(satellite[:-3] + ' C061 ' + year + ' ' + julday, size=14)
    plt.savefig(path + 'png/' + satellite + '_C061_' + year + '_' + julday + '.nc', dpi=200, alpha=True, bbox_inches='tight')

In [ ]:
def make_plot(index, levels, conc, target, fmt, cb_label):
    ax = fig.add_subplot(gs[index], projection=ccrs.PlateCarree())
    ax.set_extent([73, 135, 18, 54], crs=ccrs.PlateCarree())
    shape_feature = ShapelyFeature(
        Reader('/nfs/a68/earlacoa/shapefiles/china/china_taiwan_hongkong_macao.shp').geometries(),
        ccrs.PlateCarree(), facecolor='none')
    ax.background_patch.set_visible(False)
    ax.outline_patch.set_visible(False)
    ax.add_feature(shape_feature, edgecolor='black', linewidth=0.5)
    norm = colors.Normalize(vmin=levels[0], vmax=levels[-2])
    cmap = colors.ListedColormap(['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#bd0026','#800026'])
    im = ax.contourf(xx, yy, conc, levels, cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    plt.annotate(r'\textbf{' + chr(97 + index) + '}', xy=(0,1.05), xycoords='axes fraction', fontsize=14, weight='bold')
    plt.title(target, fontsize=14)
    sm = plt.cm.ScalarMappable(norm=norm, cmap=im.cmap)
    sm.set_array([])  
    cb = plt.colorbar(sm, fraction=0.035, norm=norm, cmap=cmap, ticks=im.levels, format=fmt)
    cb.set_label(cb_label, size=14)
    cb.ax.tick_params(labelsize=14)